## Introduction

Nina Zumel shared a good solution to the [100 Bushels of Corn](https://ninazumel.com/blog/2024-09-26-100bushels/) puzzle [here](https://ninazumel.com/blog/2024-09-26-100bushels/#the-solution).

The puzzle is:

> 100 bushes of corn are distributed to 100 people such that every man receives 3 bushels, every woman 2 bushels, and every child 1/2 a bushel. How many men, women, and children are there?

From our domain knowledge, we know that all the variables we are solving for are non-negative integers.

Dr. Zumel and I are intensely interested in how heuristics can be adapted to full solution algorithms. Some of these classic puzzles give amazing workbenches to demonstrate technique. In this case the methodology is to solve a [system of Diophantine equations](https://en.wikipedia.org/wiki/Diophantine_equation), or integer valued linear programming. Our challenge here is: can we demonstrate the methodology using code examples? In this note we give it a try.



## Solving the problem using Diophantine equations

To solve the problem using Diophantine equations we break the problem into two parts:

  1) Write down the problem as an integral matrix equation. From this we quickly observe the solutions lie in a one dimensional subspace.
  2) Find a 1 to 1 onto (reversible) mapping from the standard integers to the integral solutions of the problem, ignoring the sign constraints. This portion is solved by finding a **_Smith normal form_** for the matrix equation (a Hermite normal form will do, but there are fewer steps to demonstrate using the Smith normal form).
  3) Find the bounded interval of solutions on the line that obey our sign constraints.


  

## Starting

Let's set up Python and write down our problem.

In [1]:
# imports
import itertools
import numpy as np
import sympy as sp

from bushels_fns import is_integers, is_non_neg, right_pseudo_inverse_of_Smith_normal_form

#  uses: https://pypi.org/project/smithnormalform/
from bushels_fns import smith_normal_form

To make our matrix integral we multiply the consumption (bushel) constraints by 2. This is the same problem written slightly differently. We want integral vectors `soln >= 0`such that `a soln = b`.

In [2]:
# our linear relations
a = sp.Matrix([
    [1, 1, 1],
    [6, 4, 1]
])

a

Matrix([
[1, 1, 1],
[6, 4, 1]])

In [3]:
# our population and consumption constraints
b = sp.Matrix([100, 200])

b

Matrix([
[100],
[200]])

In this encoding the vector `soln = [#men, #women, #children]` is a proposed solution.  To be a proper solution we must have:

  * `A soln = b` (the linear relations).
  * All entries of `soln` integral (the integrality constraints).
  * `soln >= 0` (the sign constraints).

We will solve the problem by:

  1) Enforcing the linear relations.
  2) Using a matrix factorization to map into the linear subspace enforcing the integrality constraints.
  3) Bounding our subspace to impose the sign constraints.

## The linear and integrality constraints

The linear constraints are enforced by our problem set up. Let's add in a process that respects the integrality requirement by finding a mapping that is 1 to 1 and onto the integral points of interest.

To find a mapping respecting the integrality constraints we factor our matrix `a` into Smith normal form <code>a = s<sup>-1</sup> j t<sup>-1</sup></code> where:

  * `s, j, t` all integral.
  * `s, t` unimodular (have determinant equal to `+-1`).
  * `j` is in Smith normal form: initial diagonal with more zero columns.

In [4]:
# factor a as s.inv() j t.inv()
s, t, j = smith_normal_form(a)

In [5]:

assert is_integers(s)
assert np.abs(s.det()) == 1

s

Matrix([
[6, -1],
[1,  0]])

In [6]:
# confirm properties of t and display t
assert is_integers(t)
assert np.abs(t.det()) == 1

t

Matrix([
[ 1, 1,  3],
[-2, 0, -5],
[ 1, 0,  2]])

In [7]:
# confirm properties of j and display j
assert is_integers(j)
for row_i in range(j.shape[0]):
    for col_j in range(j.shape[1]):
        if row_i != col_j:
            assert j[row_i, col_j] == 0
assert j.shape == a.shape
assert s.inv() * j * t.inv() == a

j

Matrix([
[1, 0, 0],
[0, 1, 0]])

The determinants of `s` and `t` being `+-1` is the magic. This means these matrices represent invertible 1 to 1 and onto maps from the lattice of integers <code>Z<sup>3</sup></code> to itself. The matrices `s` and `t` are useful in that represent a change of variables so that our check matrix `a` is represented by the diagonal matrix `j`. In this form calculations are easy and also preserve integrality (as we are only multiplying and adding).

## The general form of the solution

We are looking for a basis for all solutions for the linear relations `a soln = b`. We assert these are all of the form <code>soln = t (j<sup>p</sup> s b + x)</code>, where <code>j<sup>p</sup></code> is a right pseudo-inverse of `j` and `x` is any integral vector such that `j x = 0`. We use `x` to parameterize all of the solutions.

We can check the above solution. 

Given:

  * <code>a = (s<sup>-1</sup> j t<sup>-1</sup>)</code>
  * <code>soln = t (j<sup>p</sup> s b + x)</code>

Then <code>a soln = (s<sup>-1</sup> j t<sup>-1</sup>) (t (j<sup>p</sup> s b + x)) = b</code>. It is just a matter of expanding the expression to confirm this indeed equals `b`.


In code this looks like the following.

In [8]:
# build right pseudo inverse of j
jp = right_pseudo_inverse_of_Smith_normal_form(j)

jp

Matrix([
[1, 0],
[0, 1],
[0, 0]])

In [9]:
# confirm on the right inverse
assert j * jp == sp.Matrix.eye(j.shape[0])

j * jp 

Matrix([
[1, 0],
[0, 1]])

In [10]:
# write down parametric portion of solution
x_vars = [sp.Symbol(f'x_{i+1}') for i in range(j.shape[1] - j.shape[0])]
x = sp.Matrix([0] * j.shape[0] + x_vars)
assert j * x == sp.Matrix([0] * j.shape[0])

x

Matrix([
[  0],
[  0],
[x_1]])

In [11]:
# write down our general solution
soln = t * (jp * s * b + x)

soln

Matrix([
[ 3*x_1 + 500],
[-5*x_1 - 800],
[ 2*x_1 + 400]])

This solution is the heart of the puzzle. It encodes the observable congruence relations: "number of children is divisible by 2", "number of women is divisible by 5", and "number of men is congruent to 3 modulo 5."

We confirm this solution obeys the linear relations, and then continue.

In [12]:
# confirm our solution obeys the linear relations
# no matter what values we have for x
assert a * soln == b

a * soln

Matrix([
[100],
[200]])

A more common way to build up the solution is to write a bit of code instead of using algebra. In my opinion it is a bit harder to check that the direct solution solves the problem (as the earlier proof by cancellation is no longer direct). But the standard direct solution is th same and is a bit easier to code.

Let's take a quick look at the "by the book" solution (found on the Wikipedia).


In [13]:
# a more direct way to get the same solution (code instead of algebra)
# the algebra above justifies this as a correct calculation
soln_direct = t * sp.Matrix(
    [(s * b)[i]/ j[i, i] for i in range(j.shape[0])] 
    + x_vars)
assert soln == soln_direct

soln_direct

Matrix([
[ 3*x_1 + 500],
[-5*x_1 - 800],
[ 2*x_1 + 400]])

Now that we have a parameterized solution that:

  * **is integral** (by the use of the integer preserving unimodular mappings or change of variables supplied by `s` and `t`)
  * **parametrizes all solutions of the linear relations** (by the introduction of the `x` symbols and a quick appeal to linear rank arguments)

all that remains is to bring in the non-negativity constraints. 

This is just a matter of bounding which values of the `x` parameters yield solutions of the correct sign.

## Enforcing the sign constraints

Now we need to know what values to substitute in for `x`. The theory tells us that all integers lead to integral solutions (and fractional values do not need to be considered). So all that remains is to find the range of `x` that obey the non-negativity constraints on the number of each type of people.

We want to find the interval (or in general the polytope) of all `x` such all three coordinates of the solution are nonnegative. Our system is linear, so the interval of non-negative solutions is bounded by where one of the solution coordinates is zero. We now find those zero crossing points by calling Sympy's solver.

In [14]:

# solve for each entry of soln being 0 in turn
pts = [sp.solve(soln[i])[0] for i in range(soln.shape[0])]

pts



[-500/3, -160, -200]

In [15]:
# show which solutions are non-negative
for pi in pts:
    si = soln.subs(x_vars[0], pi)
    print(f"x={pi}")
    print(" -> soln=")
    display(si)
    print(f" -> was non-negative: {is_non_neg(si)}")

x=-500/3
 -> soln=


Matrix([
[    0],
[100/3],
[200/3]])

 -> was non-negative: True
x=-160
 -> soln=


Matrix([
[20],
[ 0],
[80]])

 -> was non-negative: True
x=-200
 -> soln=


Matrix([
[-100],
[ 200],
[   0]])

 -> was non-negative: False


We see some of these proposed interval endpoints drive solution coordinates negative. So we keep only the ones without this problem.

In [16]:
# keep only solutions where all other entries are nonnegative
pts = [pi for pi in pts if is_non_neg(soln.subs(x_vars[0], pi))]

pts

[-500/3, -160]

And, as we are only considering integral `x`, we can sharpen this range up a bit to the following. This sort of improvement drives what are called "cutting plane methods" in integer programming.

In [17]:
# shrink interval to integer bounds
pts = [int(np.ceil(np.min(pts))), int(np.floor(np.max(pts)))]

pts

[-166, -160]

# Confirming the solutions

Now we just plug in the integers in the range `[-166, -160]` to see the exact 7 solutions to the original problem.

In [18]:
assert (pts[1] + 1 - pts[0]) == 7

for pi in range(pts[0], pts[1]+1):
    soln_x = soln.subs(x_vars[0], pi)
    assert a * soln_x == b
    assert is_non_neg(soln_x)
    assert is_integers(soln_x)
    display(soln_x)

Matrix([
[ 2],
[30],
[68]])

Matrix([
[ 5],
[25],
[70]])

Matrix([
[ 8],
[20],
[72]])

Matrix([
[11],
[15],
[74]])

Matrix([
[14],
[10],
[76]])

Matrix([
[17],
[ 5],
[78]])

Matrix([
[20],
[ 0],
[80]])

These are exactly [the solutions shown by Nina Zumel](https://ninazumel.com/blog/2024-09-26-100bushels/#the-solution). Her method can be thought of as using paper and pencil to: establish the linear relations, find the right modular relations, and then find the nonnegative integral endpoint.





## Brute force

Another computer approach that does work for small problems is brute force enumeration. Propose every non-negative integral vector with maximum entry no more than the obvious maximums, and keep the 7 that obey the `a soln = b` linear constraint checks. Fred Viole could not resist demonstrating that solution [here](https://www.linkedin.com/feed/update/urn:li:activity:7245245226536054785?commentUrn=urn%3Ali%3Acomment%3A%28activity%3A7245245226536054785%2C7245250416890773504%29&dashCommentUrn=urn%3Ali%3Afsd_comment%3A%287245250416890773504%2Curn%3Ali%3Aactivity%3A7245245226536054785%29).

In [19]:
# get (inclusive) ranges of all but last solution variable
# (eg the maximum possible number of men and of women)
ranges = [
    int(np.min([b[row_i] / a[row_i, col_j] 
                for row_i in range(a.shape[0])]))
     for col_j in range(a.shape[1] - 1)
]

ranges

[33, 50]

These naive ranges combine to the following number of combinations.

In [20]:
np.prod([r + 1 for r in ranges])

1734

We can quickly run through all of these solution proposals and limit down to correct solutions.

In [21]:
# scan through all proposed solutions, keeping correct ones
def propose_solution(v_initial):
    """propose a solution that sums to b[0] given all but one entry"""
    return [v for v in v_initial] + [b[0] - np.sum(v_initial)]

# brute force run through all combinations to inspect for solutions
brute_force_solutions = [
    tuple(pv) for v in itertools.product(*[list(range(ranges[i] + 1)) for i in range(len(ranges))])
    if (np.min(pv := propose_solution(v)) >= 0) and (a * sp.Matrix(pv) == b)
]
assert len(brute_force_solutions) == (pts[1] + 1 - pts[0])

brute_force_solutions

[(2, 30, 68),
 (5, 25, 70),
 (8, 20, 72),
 (11, 15, 74),
 (14, 10, 76),
 (17, 5, 78),
 (20, 0, 80)]

All of these methods are examples of how [so much became possible with programable calculating machines](https://win-vector.com/2009/05/12/the-joy-of-calculation/). For small problems the brute-force method merely wastes a few micro-seconds of CPU time. For large problems the earlier systematic solution method becomes important.

## Conclusion

The above may all seem "a bit magic." The reason is that these tools were designed to solve exactly these problems and to address the fundamental difficulties.

The amount of interesting mathematical structure in this puzzle is truly astounding. It allowed us to touch on quite a number of points of Diophantine equations, linear algebra, and integer programming. Problems like it are the basis of a number of deep and fascinating fields including: "diophantine equations", "lattice reduction", "lattice polytopes", "geometry of numbers", "counting integral solutions", "additive number theory", and "additive combinatorics".

I presume earlier solution methods involved a lot more direct calculation and a lot less delegation to methods. Henry Dudeney's solution is in fact just the 7 vectors without any explanation of method. I wonder if such were the Sudoku of their era. 
